## Задание 1

https://www.hackerrank.com/challenges/detect-the-email-addresses/problem?isFullScreen=true

In [ ]:
import re
txt='\n'.join([input() for k in range(int(input()))])
print(*sorted(set(re.findall(r'[_0-9a-zA-Z]+[_\.0-9a-zA-Z]*@[0-9a-zA-Z_]+[\.0-9a-zA-Z_]+[0-9a-zA-Z_]+',txt))),sep=';')

## Задание 2

https://www.hackerrank.com/challenges/detect-the-domain-name/problem?isFullScreen=true

In [2]:
import re
txt='\n'.join([input() for k in range(int(input()))])
print(*sorted(set(re.findall(r'https?://(?:ww(?:w|2)\.)?([\w\.\-]*\.[a-zA-Z]+)',txt))),sep=';')

KeyboardInterrupt: Interrupted by user

## Задание 3

Реализовать stemming, lemmatization & BoW на следующем датасете: 
https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

In [154]:
! pip install pymorphy3
! pip install pymorphy3-dicts-ru

  Obtaining dependency information for pymorphy3 from https://files.pythonhosted.org/packages/ee/53/862f7b7f3e488e5420bebd5cf59362cb175463ad3cfddd61ade15a738dc7/pymorphy3-2.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pymorphy3-dicts-ru from https://files.pythonhosted.org/packages/b0/67/469e9e52d046863f5959928794d3067d455a77f580bf4a662630a43eb426/pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/53.2 kB ? eta -:--:--
   --------------- ------------------------ 20.5/53.2 kB 320.0 kB/s eta 0:00:01
   ---------------------------------------- 53.2/53.2 kB 550.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.4 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.3/8.4 MB 3.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/8.4 MB 5.1 MB/s eta 0:00:02
   ----- -----------------------

In [159]:
import pandas as pd
import nltk
import numpy as np
from numpy import dot
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import WordPunctTokenizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy3

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\magvl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\magvl\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [148]:
data = pd.read_csv('D:\DS\DS school\labeled.csv')

In [149]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [160]:
tokenizer = WordPunctTokenizer()
snowball = SnowballStemmer(language="russian")
morph = pymorphy3.MorphAnalyzer()

In [ ]:
# возьмем первые 100 строк из датасета

In [141]:
data_tok = []
for x in data['comment'][:100]:
    data_tok.append(tokenizer.tokenize(x.lower()))

## Лемматизация и стемминг

In [162]:
data_lem_stem = []
for i in data_tok:
    st = []
    for j in i:
        st.append(snowball.stem(morph.normal_forms(j)[0]))
    data_lem_stem.append(st)

## BoW

In [163]:
def vectorize(tokens):
    vector=[]
    for w in vocab_filt:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

In [164]:
words = []
for i in data_lem_stem:
    for j in i:
        words.append(j)

In [165]:
print(f'Изначальное количество слов и символов: {len(words)}')

Изначальное количество слов и символов: 6130


In [166]:
vocab = unique(words)

In [167]:
print(f'Количество уникальных слов и символов: {len(vocab)}')

Количество уникальных слов и символов: 1958


In [168]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
special_char=[",",":"," ",";",".","?"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\magvl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [169]:
vocab_filt = []
for w in vocab: 
    if w not in stop_words and w not in special_char: 
        vocab_filt.append(w)

In [170]:
print(f'Количество уникальных слов и символов без стоп-слов и символов: {len(vocab_filt)}')

Количество уникальных слов и символов без стоп-слов и символов: 1895


## Векторизация

In [171]:
vectors = []
for i in data_lem_stem:
    vectors.append(vectorize(i))
print(f"Количество векторов: {len(vectors)}")    

Количество векторов: 100


## Наиболее схожие строки

In [172]:
matrix = []
for a in vectors:
    string = []
    for b in vectors:
        string.append(dot(a, b)/(norm(a)*norm(b)))
    matrix.append(string)
df = pd.DataFrame(matrix)
for i in range(df.shape[0]):
    df.iat[i,i] = 0
str = df.max().idxmax()
col = df.loc[str,:].idxmax()
print("Наиболее схожие строки:")
print(data_lem_stem[str])
print(data_lem_stem[col])
print(f"Косинусная мера между строками: {df.max().max()}")

Наиболее схожие строки:
['взят', 'как', 'пример', 'росс', ',', 'западноевропейск', 'стран', 'и', 'сша', '.', 'идт', 'метисац', ',', 'сознательн', 'политик', 'замещен', 'бел', 'населен', 'на', 'пришл', 'черн', '-', 'коричнев', '.', 'идт', 'создан', 'нов', 'рас', 'метис', ',', 'исламизац', 'и', 'почернен', '.', 'в', 'крупн', 'город', 'половин', 'населен', '-', 'выходец', 'из', 'ебен', 'мексик', ',', 'африк', ',', 'ближн', 'восток', ',', 'а', 'в', 'случа', 'с', 'росс', '-', 'кавказ', 'и', 'средн', 'аз', '.', 'этническ', 'ниггер', '-', 'арабск', 'гетт', 'вер', 'на', 'ху', 'закон', 'как', 'хотет', ',', 'чудовищн', 'по', 'масштаб', 'этническ', 'преступн', '.', 'говор', 'о', 'миграц', 'и', 'тем', 'бол', 'затрагива', 'тем', 'замещен', 'корен', 'населен', 'власт', 'нельз', ',', 'инач', 'бутылк', '.', 'свобод', 'слов', 'тут', 'не', 'для', 'вы', ',', 'молод', 'человек', '.', 'при', 'эт', 'говор', 'о', 'тот', ',', 'что', 'бел', 'должн', 'вымерет', ',', 'и', 'эт', 'нормальн', '-', 'можн', '.', 'бел

#### Результат по поиску схожих строк выглядит не логичным. Возможно, я где-то ошибся